(**Click the icon below to open this notebook in Colab**)

[![Open InColab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiangshiyin/machine-learning-for-actuarial-science/blob/main/2025-spring/week15/notebook/demo.ipynb)

# Introduction to NLP

## Preprocessing

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


data = "This is a simple example to demonstrate removing stopwords using NLTK."
stopWords = set(stopwords.words('english'))

In [ ]:
len(stopWords)

In [ ]:
stopwords.words('english')[:10]

In [ ]:
tokenized_data = word_tokenize(data)

In [ ]:
print(f"Original text: {data}")
print(f"Tokenized text: {"|".join(tokenized_data)}")

In [ ]:
filtered_tokenized_data = [
    w
    for w in tokenized_data
    if w not in stopWords
]
print(f"After removing stopwords: {filtered_tokenized_data}")

In [ ]:
print(f"Original text: {data}")
print(f"Tokenized text: {"|".join(tokenized_data)}")
print(f"After removing stopwords: {"|".join(filtered_tokenized_data)}")

## Feature Extraction

### Bag of Words

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Sample dataset
texts = [
    "I love this product",         # positive
    "This is amazing",             # positive
    "Very happy with the result",  # positive
    "I hate this",                 # negative
    "Worst experience ever",       # negative
    "Not satisfied at all"         # negative
]

labels = [1, 1, 1, 0, 0, 0]  # 1 = positive, 0 = negative

# 2. Convert text to bag-of-words vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# 3. Show feature names
print("Feature Names (Vocabulary):")
print(vectorizer.get_feature_names_out())


Feature Names (Vocabulary):
['all' 'amazing' 'at' 'ever' 'experience' 'happy' 'hate' 'is' 'love' 'not'
 'product' 'result' 'satisfied' 'the' 'this' 'very' 'with' 'worst']


In [20]:
X.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]])

In [21]:
X.shape

(6, 18)

### TF-IDF

In [22]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import nltk

# Download NLTK movie_reviews data
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

# Prepare dataset
docs = []
labels = []

for fileid in movie_reviews.fileids():
    docs.append(movie_reviews.raw(fileid))
    labels.append(movie_reviews.categories(fileid)[0])  # 'pos' or 'neg'

# Convert labels to binary format
y = [1 if label == 'pos' else 0 for label in labels]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(docs, y, test_size=0.2, random_state=42)

# Vectorize using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/xiangshiyin/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Accuracy: 0.8275
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.81      0.82       199
           1       0.82      0.84      0.83       201

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



In [23]:
feature_names = vectorizer.get_feature_names_out()
feature_names[:20]

array(['00', '000', '0009f', '007', '00s', '03', '04', '05', '05425',
       '10', '100', '1000', '10000', '100m', '101', '102', '103', '104',
       '105', '106'], dtype=object)

In [24]:
import pandas as pd

# Choose a sample document from the test set
sample_idx = 0
sample_vector = X_test_tfidf[sample_idx]

# Convert sparse vector to dense and create DataFrame
df_features = pd.DataFrame(
    data=sample_vector.toarray()[0],
    index=feature_names,
    columns=["tfidf"]
)

# Filter non-zero features and sort
df_nonzero = df_features[df_features.tfidf > 0].sort_values(by="tfidf", ascending=False)

# Show top 15 features by TF-IDF weight
print("\nTop TF-IDF features in sample test document:")
print(df_nonzero.head(15))


Top TF-IDF features in sample test document:
                 tfidf
bates         0.401204
annie         0.253364
caan          0.169454
kathy         0.148969
sledgehammer  0.127790
realises      0.117562
french        0.114832
nerve         0.107711
cake          0.104894
misery        0.102454
masterful     0.100301
basically     0.093828
prisoner      0.092226
arm           0.088148
tense         0.086170


### Word2Vec

#### Hand-craft implementation

In [ ]:
import numpy as np
import re
import random

# Sample corpus
corpus = "The quick brown fox jumps over the lazy dog"

# Preprocessing: Tokenization and vocabulary building
tokens = re.findall(r'\b\w+\b', corpus.lower())
vocab = set(tokens)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
vocab_size = len(vocab)

In [ ]:
word_to_idx

In [ ]:
idx_to_word

In [ ]:
tokens

In [ ]:
# Generate training data
def generate_training_data(tokens, window_size):
    training_data = []
    for idx, target_word in enumerate(tokens):
        target_idx = word_to_idx[target_word]
        context_range = list(range(max(0, idx - window_size), idx)) + \
                        list(range(idx + 1, min(len(tokens), idx + window_size + 1)))
        for context_idx in context_range:
            context_word = tokens[context_idx]
            context_word_idx = word_to_idx[context_word]
            training_data.append((target_idx, context_word_idx))
    return training_data

window_size = 2
training_data = generate_training_data(tokens, window_size)


In [ ]:
# Inspect the training data
print(f"Corpus: {corpus}")
print([
    (idx_to_word[t[0]], idx_to_word[t[1]])
    for t in training_data
])

In [ ]:
# Initialize parameters
embedding_dim = 10
W1 = np.random.randn(vocab_size, embedding_dim)
W2 = np.random.randn(embedding_dim, vocab_size)

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Training parameters
epochs = 1000
learning_rate = 0.01
num_negative_samples = 2

# Training loop
for epoch in range(epochs):
    loss = 0
    for target_idx, context_idx in training_data:
        # Positive sample
        h = W1[target_idx]
        u = np.dot(h, W2[:, context_idx])
        pred = sigmoid(u)
        error = pred - 1
        loss += -np.log(pred + 1e-7)
        # Gradients
        grad_W2 = error * h
        grad_W1 = error * W2[:, context_idx]
        # Update weights
        W2[:, context_idx] -= learning_rate * grad_W2
        W1[target_idx] -= learning_rate * grad_W1

        # Negative sampling
        negative_samples = random.sample([i for i in range(vocab_size) if i != context_idx], num_negative_samples)
        for neg_idx in negative_samples:
            u_neg = np.dot(h, W2[:, neg_idx])
            pred_neg = sigmoid(u_neg)
            error_neg = pred_neg
            loss += -np.log(1 - pred_neg + 1e-7)
            # Gradients
            grad_W2_neg = error_neg * h
            grad_W1_neg = error_neg * W2[:, neg_idx]
            # Update weights
            W2[:, neg_idx] -= learning_rate * grad_W2_neg
            W1[target_idx] -= learning_rate * grad_W1_neg
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")

In [ ]:
# Retrieve word embeddings
word_embeddings = W1

# Example: Find similar words
def find_similar(word, top_n=3):
    if word not in word_to_idx:
        print(f"'{word}' not in vocabulary.")
        return
    idx = word_to_idx[word]
    vec = word_embeddings[idx]
    similarities = []
    for i in range(vocab_size):
        if i == idx:
            continue
        sim = np.dot(vec, word_embeddings[i]) / (np.linalg.norm(vec) * np.linalg.norm(word_embeddings[i]))
        similarities.append((idx_to_word[i], sim))
    similarities.sort(key=lambda x: x[1], reverse=True)
    for word, sim in similarities[:top_n]:
        print(f"{word}: {sim:.4f}")

# Test the model
print("\nWords similar to 'fox':")
find_similar('fox')

#### With `Gensim`

In [ ]:
import gensim
from gensim.models import Word2Vec

# Sample corpus
sentences = [
    ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"],
    ["i", "love", "natural", "language", "processing"],
    ["word2vec", "is", "a", "technique", "for", "natural", "language", "processing"],
    ["the", "dog", "is", "lazy", "but", "the", "brown", "fox", "is", "quick"]
]


In [ ]:
# Initialize and train the model
model = Word2Vec(
    sentences,
    vector_size=100,  # Dimensionality of the word vectors
    window=5,         # Maximum distance between the current and predicted word
    min_count=1,      # Ignores all words with total frequency lower than this
    workers=4,        # Use these many worker threads to train the model
    sg=1              # 1 for Skip-gram; 0 for CBOW
)

In [ ]:
# Find most similar words
similar_words = model.wv.most_similar("fox", topn=3)
print(similar_words)

# Compute similarity between two words
similarity = model.wv.similarity("dog", "fox")
print(f"Similarity between 'dog' and 'fox': {similarity:.4f}")

In [ ]:
sample = """
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.


The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.


When Mr. and Mrs. Dursley woke up on the dull, gray Tuesday our story
starts, there was nothing about the cloudy sky outside to suggest that
strange and mysterious things would soon be happening all over the
country. Mr. Dursley hummed as he picked out his most boring tie for
work, and Mrs. Dursley gossiped away happily as she wrestled a screaming
Dudley into his high chair.
"""

sentences = [
    gensim.utils.simple_preprocess(sentence)
    for sentence in sample.split("\n\n")
]

In [ ]:
model = Word2Vec(
    sentences,
    vector_size=100,  # Dimensionality of the word vectors
    window=5,         # Maximum distance between the current and predicted word
    min_count=1,      # Ignores all words with total frequency lower than this
    workers=4,        # Use these many worker threads to train the model
    sg=1              # 1 for Skip-gram; 0 for CBOW
)

In [ ]:
# Find most similar words
similar_words = model.wv.most_similar("potter", topn=3)
print(similar_words)

### `GLoVE`

- https://nlp.stanford.edu/projects/glove/
- https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

In [ ]:
import gensim.downloader

In [ ]:
# All available models in gensim-data
for model in gensim.downloader.info()['models'].keys():
    print(model)

In [ ]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

In [ ]:
glove_vectors.most_similar('twitter', topn=20)

In [ ]:
glove_vectors.most_similar('president', topn=20)

In [ ]:
glove_vectors.most_similar('usa', topn=20)

In [ ]:
glove_vectors.get_vector('king')

In [ ]:
king = glove_vectors.get_vector('king')
queen = glove_vectors.get_vector('queen')
man = glove_vectors.get_vector('man')
woman = glove_vectors.get_vector('woman')

res = king - man + woman

In [ ]:
# calculate the cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(res.reshape(1, -1), queen.reshape(1, -1))
print(f"Similarity between queen and res: {similarity[0][0]}")

In [ ]:
# calculate the cosine similarity of two vectors following the linear algebra formula
import numpy as np

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
